# RedPanda Examples

In [1]:
import pandas, redpanda, sqlalchemy, sys
print("RedPanda version:", redpanda.__version__)
print("Python version:", sys.version)

RedPanda version: 0.3.0
Python version: 3.5.1 (default, May 30 2016, 16:01:04) 
[GCC 5.3.0]


## Opening a session

Open a `RedPanda` session to begin exploring data:

In [2]:
engine = redpanda.create_engine("sqlite://")
Session = redpanda.orm.sessionmaker(bind=engine)
session = Session()

Note that this is syntactic sugar for SQLAlchemy calls:

In [3]:
engine = sqlalchemy.create_engine("sqlite://")
sessionmaker = sqlalchemy.orm.sessionmaker(class_=redpanda.orm.Session,
                                           query_cls=redpanda.orm.Query)
session = sessionmaker(bind=engine)

## Creating An Example Model

Define your models using the declarative API. `RedPanda` defines one such model in the `example` module:

In [4]:
import sqlalchemy.ext.declarative
Base = sqlalchemy.ext.declarative.declarative_base()
class Widget(Base):
    id        = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    timestamp = sqlalchemy.Column(sqlalchemy.DateTime)
    name      = sqlalchemy.Column(sqlalchemy.String)
    kind      = sqlalchemy.Column(sqlalchemy.String)
    units     = sqlalchemy.Column(sqlalchemy.Integer)

    __read_sql__  = {"index_col": ["timestamp"], "parse_dates": ["timestamp"]}
    __tablename__ = "widgets"

Defining the class attribute `__read_sql__` will configure default arguments passed to `pandas.read_sql()`. 

In this case we are configuring the `widgets` table to use the `timestamp` column as an index and that we wish for `pandas` to parse this column as a `datetime` object.

In [5]:
from redpanda.example import create_widgets
create_widgets(session)

## Querying Data

Use the `frame()` method of a query to return the results as a `DataFrame`:

In [6]:
session.query(Widget).frame().head()

,id,name,kind,units
timestamp,,,,
2016-01-05 13:34:00,1,thermometer,fizzer,83
2016-01-09 22:29:00,2,diaries,bopper,93
2016-01-19 10:31:00,3,sounds,buzzer,95
2016-01-27 14:25:00,4,submarines,buzzer,5
2016-02-05 06:17:00,5,capital,bopper,30


### Querying with Filters

Use SQLAlchemy filters to refine the query:

In [7]:
session.query(Widget).filter(Widget.timestamp.between("2016-11-01", "2016-11-30")).frame()

,id,name,kind,units
timestamp,,,,
2016-11-02 16:47:00,63,nods,buzzer,89
2016-11-10 05:59:00,64,fire,fizzer,37
2016-11-18 07:49:00,65,interchanges,bopper,50
2016-11-18 20:15:00,66,mistrial,fizzer,24
2016-11-18 22:27:00,67,person,bopper,0
2016-11-28 19:59:00,68,acceptance,fizzer,41


Alternatively, use the `within()` method to query an instrumented attribute with a pandas index:

In [8]:
index = pandas.period_range("2016-11-01", "2016-11-30", freq="W")
session.query(Widget).filter(Widget.timestamp.within(index)).frame()

,id,name,kind,units
timestamp,,,,
2016-11-02 16:47:00,63,nods,buzzer,89
2016-11-10 05:59:00,64,fire,fizzer,37
2016-11-18 07:49:00,65,interchanges,bopper,50
2016-11-18 20:15:00,66,mistrial,fizzer,24
2016-11-18 22:27:00,67,person,bopper,0
2016-11-28 19:59:00,68,acceptance,fizzer,41
2016-12-01 23:51:00,69,tissues,buzzer,98


### Overriding Defaults

Pass arguments to `pandas.read_sql()` into the `frame()` method:

In [9]:
session.query(Widget).frame(index_col="id").head()

,timestamp,name,kind,units
id,,,,
1,2016-01-05 13:34:00,thermometer,fizzer,83
2,2016-01-09 22:29:00,diaries,bopper,93
3,2016-01-19 10:31:00,sounds,buzzer,95
4,2016-01-27 14:25:00,submarines,buzzer,5
5,2016-02-05 06:17:00,capital,bopper,30


## Writing DataFrame to Session

Use the `add_dataframe()` method of the `RedPanda` session to place `DataFrame` rows into the session as models:

In [10]:
from datetime import datetime
dfdata = {
    "kind": "newkind", 
    "name": "newname", 
    "units": 1000, 
    "timestamp": datetime.utcnow()}
addme = pandas.DataFrame(dfdata, index=[0])
addme

,kind,name,timestamp,units
0,newkind,newname,2016-07-16 11:47:09.228669,1000


In [11]:
session.add_dataframe(Widget, addme)
session.commit()
session.query(Widget).filter(Widget.kind=="newkind").frame()

,id,name,kind,units
timestamp,,,,
2016-07-16 11:47:09.228669,76,newname,newkind,1000
